In [26]:
from qiskit import *
import pandas as pd
from qiskit import transpile
import random
import csv
import numpy as np
from qiskit.providers.aer import AerSimulator
import time
import itertools
import random
import progressbar
import pathlib
from IPython.display import clear_output
from qiskit.tools.monitor import job_monitor

from qiskit import IBMQ, Aer
from qiskit.providers.aer.noise import NoiseModel

Gates = ['x', 'y', 'z', 'h', 'cx', 'swap']
gateCosts = [1, 1, 1, 2, 5, 11]

In [58]:
def QCtoDF(qc):
    string = qc.qasm()
    circuit = string.split(';')
    circuit = circuit[3:]
    circuit.pop(len(circuit)-1)
    #print(circuit)
    with open(r"gatesTemp.csv", 'w', newline='', encoding='UTF8') as f:
        writer = csv.writer(f)
        writer.writerow(['Gate', 'Qubit'])
        for i in range(len(circuit)):            
            circuit[i] = circuit[i].replace("\n", '')
            #print(circuit[0][1])
            temp = circuit[i].split(' ')
            #temp[1] = temp[1].split(',')
            for j in range(len(temp[1])):
                 temp[1] = temp[1].replace("q[", '')
                 temp[1] = temp[1].replace("]", '')
            if temp[0] != 'measure' and temp[0] != 'barrier' and temp[0] != 'creg':
                writer.writerow(temp)
    df = pd.DataFrame(pd.read_csv(r"gatesTemp.csv"))
    return df

In [59]:
def unique(list1):
  
    # initialize a null list
    unique_list = []
  
    # traverse for all elements
    for x in list1:
        # check if exists in unique_list or not
        if x not in unique_list:
            unique_list.append(x)
    return(unique_list)


In [60]:
def depthFinder(newQubits):
    lst = list(newQubits.items())
    #get depth
    if len(lst) > 1:
        qnum = []
        for j in range(len(lst)):
            strList = str(lst[j][0]).split(',')
            qnum.append(strList[0])
        qnum = unique(qnum)
        counts = []
        for num in qnum:
            for j in range(len(lst)):
                strList = str(lst[j][0]).split(',')
                if strList[0] == num:
                    for times in range(lst[j][1]):
                        counts.append(strList[0])
        temp = []
        for num in qnum:
            temp.append(counts.count(num))
        mx = max(temp)
        return mx 
    elif len(lst) > 0:
        return(lst[0][1])
    else:
        return(0)

## QC

In [65]:
qc = QuantumCircuit(2)
qc.x(0)
qc.cx(1, 0)
qc.z(0)
qc.h(0)
qc.x(0)
qc.y(1)

df = QCtoDF(qc)


In [66]:
print(qc)

     ┌───┐┌───┐┌───┐┌───┐┌───┐
q_0: ┤ X ├┤ X ├┤ Z ├┤ H ├┤ X ├
     └───┘└─┬─┘├───┤└───┘└───┘
q_1: ───────■──┤ Y ├──────────
               └───┘          


In [67]:
df

,Gate,Qubit
0,x,0
1,cx,"1,0"
2,z,0
3,h,0
4,x,0
5,y,1


## Depth

In [68]:
qubits = df['Qubit'].value_counts()
qubits = dict(qubits)
depth = depthFinder(qubits)

In [69]:
depth

4

## Cost

In [72]:
cost = 0
for i in df['Gate']:
    cost += int(gateCosts[Gates.index(i)])

In [73]:
cost

11

In [93]:
from qiskit import *
import pandas as pd
from qiskit import transpile
import random
import csv
import numpy as np
import time
import itertools
import random
import pathlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


Gates = ['x', 'y', 'z', 'h', 'cx', 'swap']
gateCosts = [1, 1, 1, 2, 5, 11]

def QCtoDF(qc):
    string = qc.qasm()
    circuit = string.split(';')
    circuit = circuit[3:]
    circuit.pop(len(circuit)-1)
    #print(circuit)
    with open(r"gatesTemp.csv", 'w', newline='', encoding='UTF8') as f:
        writer = csv.writer(f)
        writer.writerow(['Gate', 'Qubit'])
        for i in range(len(circuit)):            
            circuit[i] = circuit[i].replace("\n", '')
            #print(circuit[0][1])
            temp = circuit[i].split(' ')
            #temp[1] = temp[1].split(',')
            for j in range(len(temp[1])):
                 temp[1] = temp[1].replace("q[", '')
                 temp[1] = temp[1].replace("]", '')
            if temp[0] != 'measure' and temp[0] != 'barrier' and temp[0] != 'creg':
                writer.writerow(temp)
    df = pd.DataFrame(pd.read_csv(r"gatesTemp.csv"))
    return df

def unique(list1):
  
    # initialize a null list
    unique_list = []
  
    # traverse for all elements
    for x in list1:
        # check if exists in unique_list or not
        if x not in unique_list:
            unique_list.append(x)
    return(unique_list)

def depthFinder(df):
    qubits = df['Qubit'].value_counts()
    qubits = dict(qubits)
    lst = list(qubits.items())
    #get depth
    if len(lst) > 1:
        qnum = []
        for j in range(len(lst)):
            strList = str(lst[j][0]).split(',')
            qnum.append(strList[0])
        qnum = unique(qnum)
        counts = []
        for num in qnum:
            for j in range(len(lst)):
                strList = str(lst[j][0]).split(',')
                if strList[0] == num:
                    for times in range(lst[j][1]):
                        counts.append(strList[0])
        temp = []
        for num in qnum:
            temp.append(counts.count(num))
        mx = max(temp)
        return mx 
    elif len(lst) > 0:
        return(lst[0][1])
    else:
        return(0)
    
def ComplexityFinder(qc):
    df = QCtoDF(qc)
    depth = depthFinder(df)
    #Get Cost
    cost = 0
    for i in df['Gate']:
        cost += int(gateCosts[Gates.index(i)])
        
    return cost, depth

qc = QuantumCircuit(2)
qc.x(0)
qc.cx(1, 0)
qc.z(0)
qc.h(0)
qc.x(0)
qc.y(1)

#Find Cost and Depth of any given QC
cost, depth = ComplexityFinder(qc)
# print(cost, depth)
dfFull = pd.DataFrame(pd.read_csv(r"NairobiDataFull3.csv"))

for i in range(len(dfFull)):
    refCost = dfFull.loc[i, "cost"]
    refDepth = dfFull.loc[i, "depth"]
    if refCost == cost and refDepth == depth:
        set = i
        break

df = pd.read_csv(r"NairobiDataFull3.csv", skiprows = set, nrows=1)
# print(df.loc[0][0])
# print(df.loc[0])
df.columns = fields

df

,cost,depth,tqc3TT,tqc3RT,tqc3Counts,tqc3Depth,tqc3Qubits,tqc2TT,tqc2RT,tqc2Counts,...,tqc1RT,tqc1Counts,tqc1Depth,tqc1Qubits,tqc0TT,tqc0RT,tqc0Counts,tqc0Depth,tqc0Qubits,sim
0,11,4,0.019536,0.094405,"{'0110010': 443, '1110010': 9, '0010000': 23, ...",3,"{4: 3, 6: 2, 1: 1, 3: 1, 5: 1}",0.019113,0.098815,"{'0110010': 441, '0000010': 20, '1010010': 6, ...",...,0.106713,"{'0010010': 416, '0110010': 450, '1010010': 13...",3,"{5: 3, 1: 2, 0: 1, 2: 1, 4: 1}",0.010817,0.098598,"{'0110010': 457, '0010010': 449, '0100010': 14...",8,"{0: 8, 5: 4, 1: 2, 2: 1, 4: 1}","{'0110010': 509, '0010010': 515}"


In [85]:
df

,cost,depth,tqc3TT,tqc3RT,tqc3Counts,tqc3Depth,tqc3Qubits,tqc2TT,tqc2RT,tqc2Counts,...,tqc1RT,tqc1Counts,tqc1Depth,tqc1Qubits,tqc0TT,tqc0RT,tqc0Counts,tqc0Depth,tqc0Qubits,sim
0,11,4,0.019536,0.094405,"{'0110010': 443, '1110010': 9, '0010000': 23, ...",3,"{4: 3, 6: 2, 1: 1, 3: 1, 5: 1}",0.019113,0.098815,"{'0110010': 441, '0000010': 20, '1010010': 6, ...",...,0.106713,"{'0010010': 416, '0110010': 450, '1010010': 13...",3,"{5: 3, 1: 2, 0: 1, 2: 1, 4: 1}",0.010817,0.098598,"{'0110010': 457, '0010010': 449, '0100010': 14...",8,"{0: 8, 5: 4, 1: 2, 2: 1, 4: 1}","{'0110010': 509, '0010010': 515}"
